In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 1. Load Dataset
df = pd.read_csv('Titanic-Dataset.csv')

# 2. Feature Selection and Target
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

X = df[features].copy()
y = df[target]

# 3. Handle Missing Values
X['Age'] = X['Age'].fillna(X['Age'].median())
X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])

# 4. Encode Categorical Features
X = pd.get_dummies(X, columns=['Sex', 'Embarked', 'Pclass'], drop_first=True)

# 5. Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("------------------------------------------------- Bagging (Random Forest) -------------------------------------------------")
# Bagging with Random Forest
bag_model = RandomForestClassifier(n_estimators=100, random_state=42)
bag_model.fit(X_train, y_train)
bag_pred = bag_model.predict(X_test)
print("Bagging Accuracy (Random Forest):", accuracy_score(y_test, bag_pred))

print("------------------------------------------------- Boosting (XGBoost) ------------------------------------------------------")
# Boosting with XGBoost
boost_model = XGBClassifier(n_estimators=100, learning_rate=0.1, eval_metric='logloss', random_state=42)
boost_model.fit(X_train, y_train)
boost_pred = boost_model.predict(X_test)
print("Boosting Accuracy (XGBoost):", accuracy_score(y_test, boost_pred))

print("------------------------------------------------- Stacking ----------------------------------------------------------------")
# Stacking
base_models = [
    ('knn', KNeighborsClassifier(n_neighbors=3)),
    ('dt', DecisionTreeClassifier(max_depth=3))
]
meta_model = LogisticRegression(max_iter=1000)

stack_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, passthrough=True)
stack_model.fit(X_train, y_train)
stack_pred = stack_model.predict(X_test)
print("Stacking Accuracy:", accuracy_score(y_test, stack_pred))

print("\nClassification Report for Stacking:")
print(classification_report(y_test, stack_pred))


------------------------------------------------- Bagging (Random Forest) -------------------------------------------------
Bagging Accuracy (Random Forest): 0.8100558659217877
------------------------------------------------- Boosting (XGBoost) ------------------------------------------------------
Boosting Accuracy (XGBoost): 0.8268156424581006
------------------------------------------------- Stacking ----------------------------------------------------------------
Stacking Accuracy: 0.7988826815642458

Classification Report for Stacking:
              precision    recall  f1-score   support

           0       0.81      0.86      0.83       105
           1       0.78      0.72      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179

